In [1]:
%reset -f
import pandas as pd
import numpy as np
seed = 2018
np.random.seed(seed)
from wavhandler import *
from utils import *
from utils_train import *
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras import Model
from keras import backend as K
from keras import Sequential
from keras.layers import Input, Dense, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.xception import Xception

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


In [2]:
# current_model = DenseNet121
# current_model = DenseNet169
# current_model = DenseNet201
# current_model = InceptionResNetV2
# current_model = InceptionV3
# current_model = MobileNet
# current_model = NASNetLarge
# current_model = NASNetMobile
# current_model = VGG16
# current_model = VGG19
current_model = Xception

model_name = TEMP_DATADIR + 'wingbeats_' + current_model.__name__

best_weights_path = model_name + '.h5'
log_path = model_name + '.log'
monitor = 'val_acc'
batch_size = 32
epochs = 100
es_patience = 7
rlr_patience = 3

SR = 8000
N_FFT = 256
HOP_LEN = int(N_FFT / 6)
input_shape = (129, 120, 1)

In [3]:
target_names = mosquitos_6#os.listdir(DATADIR)

filenames, y = get_data(#filedir= DATADIR,
                      target_names=target_names, nr_signals=np.inf, only_names=True)
print(target_names)

X_names, y = shuffle(filenames, y, random_state = seed)
X_train, X_test, y_train, y_test = train_test_split(X_names, y, stratify = y, test_size = 0.20, random_state = seed)


100%|██████████| 6/6 [00:01<00:00,  3.80it/s]


['Ae. aegypti', 'Ae. albopictus', 'An. gambiae', 'An. arabiensis', 'C. pipiens', 'C. quinquefasciatus']


In [10]:
%%timeit
z = pd.Series(X_test).apply(lambda x: get_wingbeat_timestamp(x))

9.31 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
zz = list(map(get_wingbeat_timestamp, X_test))

9.33 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
img_input = Input(shape = input_shape)

model = current_model(input_tensor = img_input, classes = len(target_names), weights = None)

In [59]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))

    # return our model
    return model

In [60]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model1 = Sequential()
    model1.add(Dense(8, input_dim=dim, activation="relu"))
    model1.add(Dense(1, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model1.add(Dense(1, activation="linear"))

    # return our model
    return model1

mlp = create_mlp(24, regress=False)
combinedInput = concatenate([model.output, mlp.output])

In [61]:
z = Dense(6, activation='relu')(combinedInput)
z = Dense(6, activation='softmax')(z)

In [62]:
combined_model = Model(inputs=[mlp.input, model.input], outputs=z)

In [63]:
combined_model.compile(optimizer = 'adam', 
                       loss = 'categorical_crossentropy', 
                       metrics = ['accuracy'])

callbacks_list = [ModelCheckpoint(monitor = monitor,
                                filepath = best_weights_path,
                                save_best_only = True,
                                save_weights_only = True,
                                verbose = 1),
                    EarlyStopping(monitor = monitor,
                                patience = es_patience,
                                verbose = 1),
                    ReduceLROnPlateau(monitor = monitor,
                                factor = 0.1,
                                patience = rlr_patience,
                                verbose = 1),
                    CSVLogger(filename = log_path)]

In [ ]:
def train_generator(X_train, X2_train, y_train, batch_size, target_names, crop=False):
    while True:
        for start in range(0, len(X_train), batch_size):
            x_batch = []
#             x2_batch = []
            y_batch = []
#             y2_batch = []

            end = min(start + batch_size, len(X_train))
            train_batch = X_train[start:end]
            labels_batch = y_train[start:end]

            for i in range(len(train_batch)):
                data, rate = librosa.load(train_batch[i], sr = SR)
                if crop:
                    data = crop_rec(data)

                data = random_data_shift(data, u = .2)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                data = librosa.amplitude_to_db(np.abs(data))
                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)
                data = random_data_shift(data, w_limit = (-0.25, 0.25), h_limit = (-0.0, 0.0), cval = np.min(data), u = 1.0)

                # data = np.squeeze(data, axis = -1)
                # plt.imshow(data, cmap = 'gray')
                # plt.show()
                # data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
#                 x2_batch.append(get_wingbeat_timestamp(train_batch[i]))
                y_batch.append(labels_batch[i])
#                 y2_batch = 

            x_batch = np.array(x_batch, np.float32)
#             x2_batch = np.array(x2_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
#             y2_batch = np.array(y2_batch, np.float32)

            y_batch = np_utils.to_categorical(y_batch, len(target_names))

            yield [x_batch, x2_batch, y_batch]

def valid_generator(X_test, y_test, batch_size, target_names, crop=False):
    while True:
        for start in range(0, len(X_test), batch_size):
            x_batch = []
            y_batch = []

            end = min(start + batch_size, len(X_test))
            test_batch = X_test[start:end]
            labels_batch = y_test[start:end]

            for i in range(len(test_batch)):
                data, rate = librosa.load(test_batch[i], sr = SR)
                if crop:
                    data = crop_rec(data)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                data = librosa.amplitude_to_db(np.abs(data))
                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)

            y_batch = np_utils.to_categorical(y_batch, len(target_names))

            yield x_batch, y_batch

In [106]:
start = 0
crop = False
x_batch = []
#             x2_batch = []
y_batch = []
#             y2_batch = []

end = min(start + batch_size, len(X_train))
train_batch = X_train[start:end]
labels_batch = y_train[start:end]

In [107]:
for i in range(len(train_batch)):
    data, rate = librosa.load(train_batch[i], sr = SR)
    if crop:
        data = crop_rec(data)

#     data = random_data_shift(data, u = .2)

    data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
    data = librosa.amplitude_to_db(np.abs(data))
    data = np.flipud(data)

    data = np.expand_dims(data, axis = -1)
    data = random_data_shift(data, w_limit = (-0.25, 0.25), h_limit = (-0.0, 0.0), cval = np.min(data), u = 1.0)

    # data = np.squeeze(data, axis = -1)
    # plt.imshow(data, cmap = 'gray')
    # plt.show()
    # data = np.expand_dims(data, axis = -1)

    x_batch.append(data)
#                 x2_batch.append(get_wingbeat_timestamp(train_batch[i]))
    y_batch.append(labels_batch[i])

In [109]:
x_batch = np.array(x_batch, np.float32)
#             x2_batch = np.array(x2_batch, np.float32)
y_batch = np.array(y_batch, np.float32)
#             y2_batch = np.array(y2_batch, np.float32)

y_batch = np_utils.to_categorical(y_batch, len(target_names))

In [112]:
y_batch.shape

(32, 6)

In [122]:
for i, x in enumerate(X_train[:100]):
#     print(i,x)
    print(get_wingbeat_timestamp(x).hour)

18
5
21
10
14
16
14
9
0
19
12
17
19
11
21
6
14
16
22
13
15
14
20
15
16
15
14
22
16
0
17
14
19
3
4
12
17
4
22
13
11
11
19
15
11
20
19
12
12
20
13
23
19
11
17
12
9
10
2
14
13
17
17
18
23
13
16
15
10
9
0
11
22
4
23
14
6
20
15
5
18
19
0
20
19
21
17
0
19
20
14
11
13
17
14
16
22
1
22
18
